<h1 style="color:#65AE11;">Considerations for Copy/Compute Overlap</h1>

In this section you will perform host-to-device and device-to-host memory transfers in non-default streams.

<h2 style="color:#65AE11;">Objectives</h2>

By the time you complete this section you will:

* Understand how to use data chunking in service of achieving copy/compute overlap
* Learn indexing techniques that allow flexible code capable of handling arbitrary data sizes and number of streams

<h2 style="color:#65AE11;">Instructor Presentation</h2>

Please give your attention to the instructor while they present the slides.

Run the following cell to load the slide deck for this section. If you wish, you can click on "Start Slide Show" once the slides appear to view them full-screen.

In [1]:
from IPython.display import IFrame
IFrame("https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-04-V1/task1/08_copy_compute-03.pptx", 900, 640)

<h2 style="color:#65AE11;">Copy/Compute Overlap Example Code</h2>

Below are two code examples for the techniques presented above, first for when the number of entries is evenly divided by the number of streams, and second, for when this is not so.

<h3 style="color:#65AE11;">N is Evenly Divided by Number of Streams</h3>

```c
// "Simple" version where number of entries is evenly divisible by number of streams.

// Set to a ridiculously low value to clarify mechanisms of the technique.
const uint64_t num_entries = 10;
const uint64_t num_iters = 1UL << 10;

// Allocate memory for all data entries. Make sure to pin host memory.
cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc    (&data_gpu, sizeof(uint64_t)*num_entries);

// Set the number of streams.
const uint64_t num_streams = 2;

// Create an array of streams containing number of streams
cudaStream_t streams[num_streams];
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamCreate(&streams[stream]);

// Set number of entries for each "chunk". Assumes `num_entries % num_streams == 0`.
const uint64_t chunk_size = num_entries / num_streams;

// For each stream, calculate indices for its chunk of full dataset and then, HtoD copy, compute, DtoH copy.
for (uint64_t stream = 0; stream < num_streams; stream++) {

    // Get start index in full dataset for this stream's work.
    const uint64_t lower = chunk_size*stream;
    
    // Stream-indexed (`data+lower`) and chunk-sized HtoD copy in the non-default stream
    // `streams[stream]`.
    cudaMemcpyAsync(data_gpu+lower, data_cpu+lower, 
           sizeof(uint64_t)*chunk_size, cudaMemcpyHostToDevice, 
           streams[stream]);
    
    // Stream-indexed (`data_gpu+lower`) and chunk-sized compute in the non-default stream
    // `streams[stream]`.
    decrypt_gpu<<<80*32, 64, 0, streams[stream]>>>
        (data_gpu+lower, chunk_size, num_iters);
    
    // Stream-indexed (`data+lower`) and chunk-sized DtoH copy in the non-default stream
    // `streams[stream]`.
    cudaMemcpyAsync(data_cpu+lower, data_gpu+lower, 
           sizeof(uint64_t)*chunk_size, cudaMemcpyDeviceToHost, 
           streams[stream]);
}

// Destroy streams.
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamDestroy(streams[stream]);
```

<h3 style="color:#65AE11;">N is Not Evenly Divided by Number of Streams</h3>

```c
// Able to handle when `num_entries % num_streams != 0`.

const uint64_t num_entries = 10;
const uint64_t num_iters = 1UL << 10;

cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc    (&data_gpu, sizeof(uint64_t)*num_entries);

// Set the number of streams to not evenly divide num_entries.
const uint64_t num_streams = 3;

cudaStream_t streams[num_streams];
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamCreate(&streams[stream]);

// Use round-up division (`sdiv`, defined in helper.cu) so `num_streams*chunk_size`
// is never less than `num_entries`.
// This can result in `num_streams*chunk_size` being greater than `num_entries`, meaning
// we will need to guard against out-of-range errors in the final "tail" stream (see below).
const uint64_t chunk_size = sdiv(num_entries, num_streams);

for (uint64_t stream = 0; stream < num_streams; stream++) {

    const uint64_t lower = chunk_size*stream;
    // For tail stream `lower+chunk_size` could be out of range, so here we guard against that.
    const uint64_t upper = min(lower+chunk_size, num_entries);
    // Since the tail stream width may not be `chunk_size`,
    // we need to calculate a separate `width` value.
    const uint64_t width = upper-lower;

    // Use `width` instead of `chunk_size`.
    cudaMemcpyAsync(data_gpu+lower, data_cpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyHostToDevice, 
           streams[stream]);

    // Use `width` instead of `chunk_size`.
    decrypt_gpu<<<80*32, 64, 0, streams[stream]>>>
        (data_gpu+lower, width, num_iters);

    // Use `width` instead of `chunk_size`.
    cudaMemcpyAsync(data_cpu+lower, data_gpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyDeviceToHost, 
           streams[stream]);
}

// Destroy streams.
for (uint64_t stream = 0; stream < num_streams; stream++)
    cudaStreamDestroy(streams[stream]);
```

<h2 style="color:#65AE11;">Check for Understanding</h2>

Please answer the following to confirm you've learned the main objectives of this section. You can display the answers for each question by clicking on the "..." cells below the questions.

---

**What is problematic about using 3 separate non-default streams: one for all host-to-device memory transfer, one for all GPU compute, and one for all device-to-host memory transfer?**

**Answer:**

There is no guaranteed ordering between operations in different non-default streams. Issuing host-to-device transfer, GPU compute, and device-to-host transfer each in their own non-default stream could violate the constraints that GPU compute depends on the completion of host-to-device transfer, and that device-to-host transfer depends on the completion of GPU compute.

---

**Why is partitioning data into chunks an effective component of acheiving copy/compute overlap?**

**Answer:**

When we chunk our data we can...

1. Maintain correct operational order between host-to-device transfer, GPU compute, and device-to-host transfer for each chunk of data by using the same non-default stream for each of these 3 operations
2. Use different non-default streams for different chunks of data to create the possibility of overlapping copy in a given non-default stream with compute (and copy in the other direction) in another non-default stream.

---

<h2 style="color:#65AE11;">Next</h2>

Equipped with an understanding of how to acheive copy/compute overlap for an arbitrary amount of data and number of streams, you will in the next section apply your understanding to acheive copy/compute overlap in the cipher application.

Please continue to the next section: [*Exercise: Apply Copy/Compute Overlap*](../09_Exercise_Apply_Streams/Exercise_Apply_Streams.ipynb).

<h2 style="color:#65AE11;">Optional Further Study</h2>

The following are for students with time and interest to do additional study on topics related to this workshop.

* The GTC presentation [*CUDA Streams: Best Practices and Common Pitfalls*](https://on-demand.gputechconf.com/gtc/2014/presentations/S4158-cuda-streams-best-practices-common-pitfalls.pdf) will give you a review of many topics covered in this workshop, as well as some additional topics, and examples of common scenarios where the use of streams can go wrong.